<a href="https://colab.research.google.com/github/Saish31/AIML/blob/main/Quora_AutoComplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Build Quora Questions Auto-Complete Suggester</h1>

- The user can enter a question, and the code retrieves the most similar questions from the dataset using the util.semantic_search method.

- This is a symmetric search task, as the search queries have the same length and content as the questions in the corpus.

- As embeddings model, we use the SBERT model `quora-distilbert-multilingual`,
that it aligned for 100 languages. I.e., you can type in a question in various languages and it will return the closest questions in the corpus (questions in the corpus are mainly in English).

In [3]:
# ─── Prevent Colab/Jupyter from saving any widget metadata ───
from ipywidgets import Widget
Widget.widget_manager = None

In [4]:
pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [5]:
import torch

In [6]:
from sentence_transformers import SentenceTransformer,util
import os
import pandas as pd
import time

In [7]:
df=pd.read_csv('https://github.com/laxmimerit/All-CSV-ML-Data-Files-Download/raw/refs/heads/master/quora_duplicate_questions.tsv', sep='\t', nrows=20000)

In [8]:
df.shape

(20000, 4)

In [9]:
df.head()

,id,question1,question2,is_duplicate
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [10]:
corpus_sentences=list(set(df['question1'].tolist()+df['question2'].tolist()))

In [11]:
len(corpus_sentences)

37766

In [12]:
model=SentenceTransformer('quora-distilbert-multilingual')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.54k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/447 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
corpus_embeddings=model.encode(corpus_sentences, show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/1181 [00:00<?, ?it/s]

In [14]:
while True:
  query=input("Please enter your question: ")

  if query=='n':
    break

  question_embedding=model.encode(query, convert_to_tensor=True)

  hits=util.semantic_search(question_embedding, corpus_embeddings, top_k=5)
  hits=hits[0]

  for hit in hits:
    print(corpus_sentences[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))

Please enter your question: how to study?
How can I study? (Score: 0.9978)
How should I study (Score: 0.9959)
How do you study effectively? (Score: 0.9936)
How do I study a lot? (Score: 0.9903)
How can I study efficiently? (Score: 0.9901)
Please enter your question: n
